# 📘 Jupyter Notebook: Applying Chain of Thought (CoT) Prompting with OpenAI API

---

## ✅ Objective

Learn how to apply **Chain of Thought (CoT) prompting** using **OpenAI's Chat API** to improve reasoning performance in complex questions.

---

## 🧠 What is Chain of Thought (CoT) Prompting?

Chain of Thought prompting is a technique where the model is encouraged to **"think out loud"** by generating intermediate reasoning steps before answering. This improves accuracy in tasks that require logic, calculation, or multi-step reasoning.

---


In [6]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install gradio

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [8]:
# Import required libraries
import os
from openai import OpenAI

# Set API key (use your own)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name =  "gpt-3.5-turbo"


🔍 Step 1: Define Your Questions

In [9]:
questions = [
    "ถ้ารถไฟ A ออกจากสถานีเวลา 3 โมงเย็นด้วยความเร็วเฉลี่ย 80 กม./ชม. และรถไฟ B ออกจากสถานีเดียวกันในทิศทางเดียวกันเวลา 3 โมงครึ่งด้วยความเร็ว 100 กม./ชม. รถไฟ B จะใช้เวลาเท่าไรจึงจะไล่ทันรถไฟ A และทั้งสองจะพบกันที่ระยะทางเท่าไรจากสถานี?",
    "จอห์นมีแอปเปิ้ลจำนวนหนึ่ง เมื่อวานเขาซื้อเพิ่ม 40% ของจำนวนที่มีอยู่เดิม วันนี้เขาขายไป 25% ของแอปเปิ้ลทั้งหมดที่เขามีอยู่ตอนนี้ ถ้าตอนนี้เขามีแอปเปิ้ลเหลือ 21 ลูก เดิมทีจอห์นมีแอปเปิ้ลกี่ลูก?",
    "ในกล่องใบแรกมีลูกบอลสีแดง 3 ลูก สีน้ำเงิน 2 ลูก และสีเขียว 1 ลูก ในกล่องใบที่สองมีลูกบอลสีแดง 2 ลูก สีน้ำเงิน 3 ลูก และสีเขียว 2 ลูก หากสุ่มหยิบลูกบอล 1 ลูกจากแต่ละกล่อง ความน่าจะเป็นที่จะได้ลูกบอลสีเดียวกันจากทั้งสองกล่องคือเท่าไร?",
    "หนังสือชุดหนึ่งมี 3 เล่ม ราคาเล่มแรก 120 บาท เล่มที่สองแพงกว่าเล่มแรก 15% และเล่มที่สามถูกกว่าเล่มที่สอง 20% หากซื้อหนังสือทั้งชุดและได้รับส่วนลดเพิ่มอีก 5% สำหรับยอดรวมทั้งหมด จะต้องจ่ายเงินทั้งหมดเท่าไร?",
    "ชาวนามีวัว ไก่ และแกะ จำนวนวัวเป็น 2 เท่าของจำนวนไก่ และจำนวนแกะเป็นครึ่งหนึ่งของจำนวนวัว ถ้าสัตว์ทั้งหมดมีขารวมกัน 124 ขา ชาวนามีสัตว์แต่ละชนิดกี่ตัว?",
    "ผลรวมของเลขคี่ 50 จำนวนแรกคือเท่าไร? และถ้าเรานำผลรวมนั้นไปหารด้วยผลรวมของเลขคู่ 25 จำนวนแรก จะได้ผลลัพธ์เท่าไร?",
    "ถ้าวันนี้เป็นวันอังคารที่ 25 มิถุนายน 2567 อีก 100 วันข้างหน้าจะเป็นวันอะไรในสัปดาห์และในเดือนใดของปี?",
    "สี่เหลี่ยมผืนผ้าหนึ่งมีความยาว 10 หน่วย และความกว้าง 4 หน่วย ถ้าเพิ่มความยาวขึ้น 20% และลดความกว้างลง 25% พื้นที่ของสี่เหลี่ยมผืนผ้าใหม่จะเป็นเท่าไร และคิดเป็นกี่เปอร์เซ็นต์ของพื้นที่เดิม?",
    "ถ้า 2x + 3y = 17 และ 4x - y = 9 ค่าของ x * y คือเท่าไร?",
    "รถยนต์ A วิ่งด้วยความเร็ว 60 กิโลเมตรต่อชั่วโมงเป็นเวลา 2.5 ชั่วโมง จากนั้นเพิ่มความเร็วเป็น 80 กิโลเมตรต่อชั่วโมงเป็นเวลา 1 ชั่วโมง ในขณะที่รถยนต์ B วิ่งด้วยความเร็วคงที่ตลอด 3.5 ชั่วโมง หากรถยนต์ A และ B ออกเดินทางพร้อมกันและมาถึงปลายทางพร้อมกัน รถยนต์ B วิ่งด้วยความเร็วเฉลี่ยเท่าไร?",
    "นาฬิกาเรือนหนึ่งเดินเร็วไป 5 นาทีทุกๆ 24 ชั่วโมง หากนาฬิกาตั้งตรงตอนเที่ยงวันของวันจันทร์ เมื่อเวลา 5 โมงเย็นของวันพฤหัสบดี นาฬิกาเรือนนี้จะบอกเวลาจริงว่ากี่โมง?",
    "กล่องขนาดใหญ่หนัก 2 กิโลกรัม ภายในมีกล่องขนาดกลาง 3 กล่อง แต่ละกล่องขนาดกลางหนัก 1 กิโลกรัม และในแต่ละกล่องขนาดกลางมีกล่องขนาดเล็ก 2 กล่อง แต่ละกล่องขนาดเล็กหนัก 0.5 กิโลกรัม น้ำหนักรวมของกล่องทั้งหมดนี้คือเท่าไร?",
    "สมชายมีเงิน 1,000 บาท เขาต้องการซื้อขนม 3 ชนิด: ขนมปังราคาชิ้นละ 25 บาท, คุกกี้ราคาชิ้นละ 40 บาท, และแซนด์วิชราคาชิ้นละ 60 บาท หากเขาต้องการซื้อขนมแต่ละชนิดอย่างน้อย 3 ชิ้น และใช้เงินให้ใกล้เคียง 1,000 บาทมากที่สุด โดยที่จำนวนชิ้นรวมทั้งหมดต้องไม่เกิน 20 ชิ้น เขาควรซื้อขนมแต่ละชนิดกี่ชิ้นจึงจะเหลือเงินน้อยที่สุด?",
    "รถคันหนึ่งเติมน้ำมันเต็มถัง 50 ลิตร วิ่งด้วยอัตราสิ้นเปลืองน้ำมัน 10 กิโลเมตรต่อลิตรสำหรับ 70% แรกของถัง และ 12 กิโลเมตรต่อลิตรสำหรับน้ำมันที่เหลือ ถ้ารถคันนี้วิ่งจากเมือง A ไปเมือง B เป็นระยะทาง 450 กิโลเมตร จะเหลือน้ำมันกี่ลิตรเมื่อถึงเมือง B?",
    "นักเรียน 15 คนต้องการแบ่งขนม 200 ชิ้นเท่าๆ กัน แต่มีนักเรียน 3 คนที่ป่วยไม่ได้มาโรงเรียน ขนมที่เหลือจะถูกแบ่งให้นักเรียนที่เหลือเท่าๆ กัน นักเรียนแต่ละคนที่มาโรงเรียนจะได้รับขนมเพิ่มขึ้นจากเดิมกี่ชิ้น?"
]

🧠 Step 2: Use Chain of Thought (CoT) Prompting

In [10]:
def ask_with_cot(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that thinks step by step."},
        {"role": "user", "content": f"{question}\nLet's think step by step."}
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return response.choices[0].message.content

# Example
print(ask_with_cot(questions[1]))


Step 1: เมื่อวานเขาซื้อเพิ่ม 40% ของจำนวนที่มีอยู่เดิม
- ค่าเพิ่มขึ้น = 40% ของจำนวนที่มีอยู่เดิม
- ค่าเดิม = 100%
- รวมค่าที่มีเดิมและค่าเพิ่มขึ้น = 100% + 40% = 140%

Step 2: วันนี้เขาขายไป 25% ของแอปเปิ้ลทั้งหมดที่เขามีอยู่ตอนนี้
- เหลืออยู่ = 100% - 25% = 75%

Step 3: ถ้าตอนนี้เขามีแอปเปิ้ลเหลือ 21 ลูก
- ค่าที่เหลือ = 75%
- ค่าเหลือ = 21 ลูก

Step 4: ดังนั้นเราสามารถคำนวณจำนวนแอปเปิ้ลเดิมที่จอห์นมีได้ดังนี้
- 21 ลูก = 75%
- 1 ลูก = 75% / 21
- ทั้งหมด = 100% = 1 ลูก / 75% x 100%

Step 5: นำจำนวนแอปเปิ้ลที่ได้จากขั้นตอนก่อนหน้านี้มาคิดต่อ
- จำนวนแอปเปิ้ลที่จอห์นมีเดิม = 1 ลูก / (75% / 21) x 100%

ดังนั้น หาก จอห์นมีแอปเปิ้ล 80 ลูก เดิม



🔁 Step 3: Without Chain of Thought (Baseline)

In [11]:
def ask_without_cot(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return response.choices[0].message.content

# Example
print(ask_without_cot(questions[1]))


เราจะใช้ขั้นตอนการคำนวณเพื่อหาจำนวนแอปเปิ้ลที่เดิมที่จอห์นมีอยู่

1. มาก่อนวาน:
- จำนวนแอปเปิ้ลที่จอห์นซื้อเพิ่ม = 40% ของจำนวนที่มีอยู่เดิม
- จำนวนที่จอห์นซื้อเพิ่มเท่ากับ 0.4x (เมื่อ x คือจำนวนแอปเปิ้ลที่มีอยู่เดิม)
- จำนวนที่มีอยู่หลังจากซื้อเพิ่ม = x + 0.4x = 1.4x

2. วันนี้:
- จำนวนแอปเปิ้ลที่จอห์นขาย = 25% ของจำนวนทั้งหมดที่มี
- จำนวนที่จอห์นขาย = 0.25(1.4x) = 0.35x

3. จำนวนแอปเปิ้ลที่เหลือ = 21 ลูก

จะได้ว่า 0.65x = 21
x = 21 / 0.65 ≈ 32.31

ดังนั้น จำนวนแอปเปิ้ลที่จอห์นมีเดิมไว้คือ 32 ลูก


📊 Step 4: Compare Outputs

In [12]:
for q in questions:
    print(f"🟡 Question: {q}")
    print("🔵 Without CoT:", ask_without_cot(q))
    print("\n")
    print("🟢 With CoT:", ask_with_cot(q))
    print("-" * 80)


🟡 Question: ถ้ารถไฟ A ออกจากสถานีเวลา 3 โมงเย็นด้วยความเร็วเฉลี่ย 80 กม./ชม. และรถไฟ B ออกจากสถานีเดียวกันในทิศทางเดียวกันเวลา 3 โมงครึ่งด้วยความเร็ว 100 กม./ชม. รถไฟ B จะใช้เวลาเท่าไรจึงจะไล่ทันรถไฟ A และทั้งสองจะพบกันที่ระยะทางเท่าไรจากสถานี?
🔵 Without CoT: ให้พิจารณาจากข้อมูลดังนี้

- รถไฟ A ออกจากสถานีเวลา 3 โมงเย็น ด้วยความเร็วเฉลี่ย 80 กม./ชม.
- รถไฟ B ออกจากสถานีเวลา 3 โมงครึ่ง ด้วยความเร็ว 100 กม./ชม.

เราต้องหาเวลาที่รถไฟ B จะไล่ทันรถไฟ A ซึ่งเป็นระยะเวลาที่ทั้งสองรถจะพบกันที่ระยะทางเท่ากัน

ให้ x คือ เวลาที่ใช้ให้รถไฟ B ไล่ทันรถไฟ A

โดยใช้สูตรทางคณิตศาสตร์
ระยะทาง = ความเร็ว * เวลา

รถไฟ A ไป x ชั่วโมง จะเข้าพบกับรถไฟ B ที่ระยะทาง 80x กม.
รถไฟ B ไป x ชั่วโมง จะเข้าพบกับรถไฟ A ที่ระยะทาง 100x กม.

เนื่องจากรถไฟ A ไปได้ 80x กิโลเมตร และรถไฟ B ไปได้ 100x กิโลเมตร เราสามารถตั้งสมการได้ว่า
80x = 100x

จากสมการนี้ ลบ 80x ออกจากทั้ง 2 คู่ จะได้
20x = 0

x = 0

ดังนั้น รถไฟ B จะไล่ทันรถไฟ A ในเวลา x = 0 ชั่วโมง หรือหมายถึง รถไฟ B ได้เข้าพบกันกับรถไฟ A ที่ระยะทาง 0 กิโลเมตร นั่นหมายค